In [2]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import json
from datasets import Dataset, DatasetDict

In [10]:
def json_to_DatasetDict(file_name):
    # read the json file
    with open(f"{file_name}.json", "r") as json_file:
        data = json.load(json_file)

    # extracting element from the data variable
    text = data["annotations"][0][0]
    tags = data["annotations"][0][1]["entities"]
    classes = data["classes"]

    classes_dict = {}
    for i in range(len(classes)):
        classes_dict[classes[i]] = i

    tokens = []
    named_tags = []

    for tag in tags:
        tokens.append(text[tag[0]:tag[1]])
        named_tags.append(tag[2])

    anno_dict = dict(zip(tokens, named_tags))

    all_tokens = text.split(" ")
    named_labels = []
    numbered_labels = []

    for token in all_tokens:
        if token not in anno_dict:
            named_labels.append("NONE")
        else:
            named_labels.append(anno_dict[token])

    for label in named_labels:
        numbered_labels.append(classes_dict[label])


    # dividing the tokens into 10 token array
    all_tokens_array = []
    numbered_labels_array = []

    temp1 = []
    temp2 = []
    for i in range(len(all_tokens)):
        temp1.append(all_tokens[i])
        temp2.append(numbered_labels[i])
        if i%10 == 0 and i != 0:
            all_tokens_array.append(temp1)
            numbered_labels_array.append(temp2)
            temp1 = []
            temp2 = []
    all_tokens_array.append(temp1)
    numbered_labels_array.append(temp2)


    # packing necessary elements together to make the dict
    # train, val, test division
    list_len = len(all_tokens_array)
    train_split = int(list_len*0.8)
    val_and_test_split = int(list_len*0.1)

    data_dict_train = {
    "tokens" : all_tokens_array[0:train_split],
    "labels" : numbered_labels_array[0:train_split]
    }

    data_dict_val = {
        "tokens" : all_tokens_array[train_split:train_split+val_and_test_split],
        "labels" : numbered_labels_array[train_split:train_split+val_and_test_split]
    }

    data_dict_test = {
        "tokens" : all_tokens_array[train_split+val_and_test_split:],
        "labels" : numbered_labels_array[train_split+val_and_test_split:]
    }

    dataset_train = Dataset.from_dict(data_dict_train)
    dataset_val = Dataset.from_dict(data_dict_val)
    dataset_test = Dataset.from_dict(data_dict_test)

    dataset_dict = DatasetDict({"train": dataset_train, "validate": dataset_val, "test": dataset_test})

    return dataset_dict

In [15]:
data_set_dict = json_to_DatasetDict("annotations")
print(data_set_dict)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 200
    })
    validate: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 25
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 25
    })
})
